NTKL TEST

In [59]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to C:\Users\Larissa
[nltk_data]     Lorenzi\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [60]:
file_path = 'df5.csv'
df = pd.read_csv(file_path)

sia = SentimentIntensityAnalyzer()


In [61]:
sia.polarity_scores('I am so happy!')

{'neg': 0.0, 'neu': 0.318, 'pos': 0.682, 'compound': 0.6468}

In [62]:
sia.polarity_scores('I hate it.')

{'neg': 0.787, 'neu': 0.213, 'pos': 0.0, 'compound': -0.5719}

In [63]:
example = df['message_trans'][5]
print(example)

you can leave it, your help is of little value. system to give the information that there is already a user with one of the informed data. i have already informed several times what is happening but it doesn't seem to be. terrible service. it always repeats the same thing. so,


In [64]:
sia.polarity_scores(example)

{'neg': 0.098, 'neu': 0.839, 'pos': 0.063, 'compound': -0.4304}

In [65]:
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['message_trans']
    myid = row['chat_id']
    res[myid] = sia.polarity_scores(text)

  0%|          | 0/30 [00:00<?, ?it/s]

In [66]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'chat_id'})
vaders = vaders.merge(df, how='left')

In [67]:
vaders.head(2)

,chat_id,neg,neu,pos,compound,message,user,created_at,message_trans
0,14VVFZN7R80UNPVA8Q04,0.0,0.774,0.226,0.8860,Empresa Preciso cadastrsr outra empresan __aud...,Seoneco 🐸,2023-11-27T13:20:31.515216-03:00,company i need to register another company __a...
1,1DP3MDANB0OGQKASW1OM,0.0,0.919,0.081,0.5859,O cadastro já deu certo agora só falta pedir a...,Welder Vendedor,2024-01-02T12:35:00.727364-03:00,"the registration has already been successful, ..."


In [68]:
# analyse compound column value and create a new column with the result considering that if the value is > 0.05 the message is positive, if it is < -0.05 the message is negative and if it is between -0.05 and 0.05 the message is neutral

vaders.loc[vaders['compound'] >= -0.05, 'vaders_sentiment'] = 'positive'
vaders.loc[vaders['compound'] <= -0.05, 'vaders_sentiment'] = 'negative'
vaders.loc[(vaders['compound'] >= -0.05) & (vaders['compound'] <= 0.05), 'vaders_sentiment'] = 'neutral'

vaders.head()

,chat_id,neg,neu,pos,compound,message,user,created_at,message_trans,vaders_sentiment
0,14VVFZN7R80UNPVA8Q04,0.0,0.774,0.226,0.8860,Empresa Preciso cadastrsr outra empresan __aud...,Seoneco 🐸,2023-11-27T13:20:31.515216-03:00,company i need to register another company __a...,positive
1,1DP3MDANB0OGQKASW1OM,0.0,0.919,0.081,0.5859,O cadastro já deu certo agora só falta pedir a...,Welder Vendedor,2024-01-02T12:35:00.727364-03:00,"the registration has already been successful, ...",positive
2,1OHYT505S3V28ELHD28U,0.0,1.000,0.000,0.0000,Mais eu sou uma criança Como faço Eu sou uma c...,Tulio undefined,2023-11-30T11:02:34.54443-03:00,"desculpe, mas o texto fornecido é muito confus...",neutral
3,25472dae-41be-4db5-b465-92b3ee0f285d,0.0,0.780,0.220,0.8670,"Meu salário é 2000,00 e vou tirar 10 dias de f...",Thaissa Delgado,2023-12-04T19:42:38.730139-03:00,my salary is 2000.00 and i'm going to take 10 ...,positive
4,2S8DYFBPKE8253H7EFPH,0.0,0.774,0.226,0.8738,O pagamento em 12x para o cliente tem juros? E...,Cidadedossonhosmoveis undefined,2023-12-07T14:08:58.011084-03:00,does the 12x payment for the customer have int...,positive


In [69]:
#selecione as linhas que deram negativo

vaders[vaders['vaders_sentiment'] == 'negative']

,chat_id,neg,neu,pos,compound,message,user,created_at,message_trans,vaders_sentiment
5,3HX5TD1K48QBH4IDESXZ,0.098,0.839,0.063,-0.4304,"pode deixar, sua ajuda muito é de pouca avalia...",Fernando Bronté,2023-12-13T11:02:26.117422-03:00,"you can leave it, your help is of little value...",negative
25,UL8J37JJ1MCYVJZOOBXJ,0.059,0.941,0.000,-0.2960,__video__ __image__ __image__ __image__ __imag...,Ged Vendas E Serviços,2024-01-01T08:35:39.067318-03:00,"como um tradutor, eu não consigo analisar imag...",negative


In [70]:
#conte quantos sentment foram positivos, negativos e neutros

vaders['vaders_sentiment'].value_counts()

vaders_sentiment
positive    24
neutral      4
negative     2
Name: count, dtype: int64

ROBERTA TEST

In [71]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [72]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [73]:
example2 = 'It was a great day!'
print(example2)

It was a great day!


In [74]:
encoded_text = tokenizer(example, return_tensors="pt")
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'roberta_neg': scores[0],
    'roberta_neu': scores[1],
    'roberta_pos': scores[2]
}

print(scores_dict)

{'roberta_neg': 0.86941254, 'roberta_neu': 0.11667889, 'roberta_pos': 0.01390859}


In [75]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors="pt")
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg': scores[0],
        'roberta_neu': scores[1],
        'roberta_pos': scores[2]
    }
    return scores_dict

In [76]:
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    try:    
        text = row['message_trans']
        myid = row['chat_id']
        roberta_results = polarity_scores_roberta(text)
        res[myid] = roberta_results
    except RuntimeError:
        print(f'Broke for chat_id {chat_id}')

  0%|          | 0/30 [00:00<?, ?it/s]

In [77]:
roberta = pd.DataFrame(res).T
roberta = roberta.reset_index().rename(columns={'index': 'chat_id'})
roberta = roberta.merge(df, how='left')

In [78]:
roberta.head(2)

,chat_id,roberta_neg,roberta_neu,roberta_pos,message,user,created_at,message_trans
0,14VVFZN7R80UNPVA8Q04,0.12992,0.732714,0.137366,Empresa Preciso cadastrsr outra empresan __aud...,Seoneco 🐸,2023-11-27T13:20:31.515216-03:00,company i need to register another company __a...
1,1DP3MDANB0OGQKASW1OM,0.06929,0.387703,0.543006,O cadastro já deu certo agora só falta pedir a...,Welder Vendedor,2024-01-02T12:35:00.727364-03:00,"the registration has already been successful, ..."


In [79]:
#crie uma coluna que mostre o sentimento de acordo com o resultado do modelo roberta, considerando que se o valor do roberta_neg for maior que o roberta_pos e o roberta_neu, o sentimento é negativo. Se o valor do roberta_pos for maior que o roberta_neg e que o roberta_neu, o sentimento é positivo. E se o valor do roberta_neu for maior que o roberta_neg e que o roberta_pos, o sentimento é neutro.

roberta.loc[(roberta['roberta_neg'] > roberta['roberta_neu']) & (roberta['roberta_neg'] > roberta['roberta_pos']), 'sentiment_roberta'] = 'negative'
roberta.loc[(roberta['roberta_pos'] > roberta['roberta_neg']) & (roberta['roberta_pos'] > roberta['roberta_neu']), 'sentiment_roberta'] = 'positive'
roberta.loc[(roberta['roberta_neu'] > roberta['roberta_neg']) & (roberta['roberta_neu'] > roberta['roberta_pos']), 'sentiment_roberta'] = 'neutral'

roberta.head(5)

,chat_id,roberta_neg,roberta_neu,roberta_pos,message,user,created_at,message_trans,sentiment_roberta
0,14VVFZN7R80UNPVA8Q04,0.129920,0.732714,0.137366,Empresa Preciso cadastrsr outra empresan __aud...,Seoneco 🐸,2023-11-27T13:20:31.515216-03:00,company i need to register another company __a...,neutral
1,1DP3MDANB0OGQKASW1OM,0.069290,0.387703,0.543006,O cadastro já deu certo agora só falta pedir a...,Welder Vendedor,2024-01-02T12:35:00.727364-03:00,"the registration has already been successful, ...",positive
2,1OHYT505S3V28ELHD28U,0.110096,0.800363,0.089541,Mais eu sou uma criança Como faço Eu sou uma c...,Tulio undefined,2023-11-30T11:02:34.54443-03:00,"desculpe, mas o texto fornecido é muito confus...",neutral
3,25472dae-41be-4db5-b465-92b3ee0f285d,0.071467,0.870992,0.057541,"Meu salário é 2000,00 e vou tirar 10 dias de f...",Thaissa Delgado,2023-12-04T19:42:38.730139-03:00,my salary is 2000.00 and i'm going to take 10 ...,neutral
4,2S8DYFBPKE8253H7EFPH,0.007801,0.143044,0.849155,O pagamento em 12x para o cliente tem juros? E...,Cidadedossonhosmoveis undefined,2023-12-07T14:08:58.011084-03:00,does the 12x payment for the customer have int...,positive


In [80]:
#conte quantos sentment foram positivos, negativos e neutros

roberta['sentiment_roberta'].value_counts()

sentiment_roberta
neutral     23
negative     4
positive     3
Name: count, dtype: int64

In [81]:
# mostre os negativos

roberta[roberta['sentiment_roberta'] == 'negative']

,chat_id,roberta_neg,roberta_neu,roberta_pos,message,user,created_at,message_trans,sentiment_roberta
5,3HX5TD1K48QBH4IDESXZ,0.869413,0.116679,0.013909,"pode deixar, sua ajuda muito é de pouca avalia...",Fernando Bronté,2023-12-13T11:02:26.117422-03:00,"you can leave it, your help is of little value...",negative
13,DT21UXHHZXMECABELPUI,0.595047,0.357828,0.047126,Ok Vou dar uma verificada! Difícil de acredita...,Fernando undefined,2023-12-14T09:39:56.224283-03:00,"ok, i'll check it out! hard to believe, the bu...",negative
17,N4QTWLZ6T43KS48FWH7F,0.481320,0.444476,0.074204,Trabalha com blue Qual fábrica de bike Normal ...,Fabio Sotero -magnum Tires,2023-12-05T17:46:42.463539-03:00,this text seems to be a bit disjointed and lac...,negative
23,R1C3FDGJVII4ZNO3DZLC,0.501066,0.425110,0.073824,Ok Vou aguardar o recebimento do e mail Agrade...,Munaretto Samuel,2023-12-19T11:53:58.295897-03:00,"ok, i will wait for the email receipt. we appr...",negative


OPENAI TEST

In [82]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [83]:
def sentiment_analysis(text):
    prompt = f"""Você é treinado para analisar e detectar o sentimento dos clientes que foram atendidos pelo atendimento do nosso chatbot.
    Somos uma fintech e queremos vender nossos serviços financeiros, desde a assinatura da nossa conta digital até a venda das nossas maquininhas de cartão para os clientes façam suas vendas conosco.
    Você deve considerar isso ao analisar as respostas dos clientes, e identificar se eles estão satisfeitos com o atendimento e com a nossa empresa.
    Você vai ler um compilado das respostas dos clientes em uma conversa e vai fazer uma análise das palavras separadamente e do contexto do texto, determinando se o sentimento desse texto é: Positivo, Negativo ou Neutro.
    Lembre que positivo é o sentimento de satisfação, alegria, felicidade, euforia, etc. Negativo é o sentimento de tristeza, raiva, frustração, etc. E neutro é o sentimento de indiferença, neutralidade, etc.
    Além disso, quando o cliente repetir mais de uma vez alguma frase, também pode ser um sinal de sentimento negativo, e se ele as respostas dele não parecem com uma conversa normal, também pode ser um sinal de sentimento negativo, porque quer dizer que ele cansou e abandonou a conversa.
    Por outro aldo, se o texto de respostas dele tiver palavras de agradecimento, um aceite dos nossos serviços ou palavras positivas, o sentimento é positivo.
    O sentimento neutro é somente para quando o cliente só perguntou sobre os serviços da empresa, sem demonstrar interesse ou insatisfação.
    Se não tiver certeza de uma resposta, você pode dizer "Não tenho certeza" e recomendar a revisão do usuário manualmente.
    {text}"""

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "Você é um assistente prestativo."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=60,
            temperature=0
        )

        sentiment = response.choices[0].message['content'].strip().lower()
        return sentiment
    except Exception as e:
        print("Ocorreu um erro:", e)
        return "Erro"

In [84]:
# Example usage
input_text = "Estou feliz com o produto!"
sentiment = sentiment_analysis(input_text)
print(input_text, ": Sentiment is:", sentiment)

Estou feliz com o produto! : Sentiment is: o sentimento deste texto é positivo. o cliente expressou satisfação com o produto, o que indica que ele está feliz com o serviço fornecido.


In [106]:
prompt = df

In [107]:
prompt.head(1)

,chat_id,message,user,created_at,message_trans,prompt_sentiment
0,14VVFZN7R80UNPVA8Q04,Empresa Preciso cadastrsr outra empresan __aud...,Seoneco 🐸,2023-11-27T13:20:31.515216-03:00,company i need to register another company __a...,a análise do sentimento desse texto é neutra. ...


In [87]:
prompt['prompt_sentiment'] = prompt['message'].apply(sentiment_analysis)

In [88]:
#mostre as linhas que contém a palavra "positivo" no texto da coluna sentiment

prompt[prompt['prompt_sentiment'].str.contains('positivo')]

,chat_id,message,user,created_at,message_trans,prompt_sentiment
4,2S8DYFBPKE8253H7EFPH,O pagamento em 12x para o cliente tem juros? E...,Cidadedossonhosmoveis undefined,2023-12-07T14:08:58.011084-03:00,does the 12x payment for the customer have int...,"analisando o texto, o sentimento geral parece ..."
7,465f7b9f-cf21-4d63-8faf-6f9b253c971a,qual o link para matricular na totalpass? Qual...,Larissa Feitosa,2023-11-29T14:43:00.202194-03:00,what is the link to enroll in totalpass? what ...,o sentimento expresso neste texto parece ser n...
8,4JH06S7FECE1IXT73JOP,Como me tornar fornecedor da Blu? Parceiros bl...,Delaine Baruff,2023-12-19T09:24:47.056425-03:00,how do i become a supplier for blu? blu partne...,a análise do sentimento desse texto é neutra. ...
10,6U66WQYCNCKQ2GANVFSG,A minha estar cobrando Taxa de aluguel da maqu...,Irene🦁 5598985529135,2023-12-13T16:54:05.515712-03:00,my account is charging a rental fee for the ma...,o sentimento desse texto parece ser neutro. o ...
11,AARXFUPLMN15SVO8GEGT,O cadastro Como faço pra saber se foi aprovado...,Fabiano Mesquista,2023-12-05T14:42:48.06681-03:00,the registration how do i know if it was appro...,a análise do sentimento desse texto é neutra. ...
19,OPVO3OQOFO8P8I0FU43M,Taxas link de pagamento Depois dos 6 meses qua...,Mania De Óculos - João Victor,2023-12-05T15:50:39.318651-03:00,payment link fees after 6 months what is the f...,a análise do sentimento desse texto é neutro. ...
23,R1C3FDGJVII4ZNO3DZLC,Ok Vou aguardar o recebimento do e mail Agrade...,Munaretto Samuel,2023-12-19T11:53:58.295897-03:00,"ok, i will wait for the email receipt. we appr...",a análise do sentimento desse texto é neutra. ...
25,UL8J37JJ1MCYVJZOOBXJ,__video__ __image__ __image__ __image__ __imag...,Ged Vendas E Serviços,2024-01-01T08:35:39.067318-03:00,"como um tradutor, eu não consigo analisar imag...",a análise do sentimento deste texto é neutro. ...
28,XUOX2WD5ORMADR46Y4IV,Vou procurar me informar melhor depois entro e...,Lucas undefined,2023-12-04T12:09:22.459087-03:00,i'll seek to inform myself better then i'll ge...,a análise do sentimento desse texto é neutra. ...
29,YWTMATI67F3RM1Z328ZB,Então enquanto não chega a maquininha posso es...,Ana undefined,2023-12-01T13:38:01.716098-03:00,"so while the machine doesn't arrive, can i be ...",o sentimento do texto é neutro. o cliente está...


In [108]:
#crie uma nova coluna que procura as palavras positivo ou positiva, negativo ou negativa ou neutro ou neutra e palavras parecidas com essas no texto e diz simplesmente que é positive, negative ou neutral

prompt.loc[prompt['prompt_sentiment'].str.contains('positivo'), 'prompt_sentiment2'] = 'positive'
prompt.loc[prompt['prompt_sentiment'].str.contains('positiva'), 'prompt_sentiment2'] = 'positive'
prompt.loc[prompt['prompt_sentiment'].str.contains('negativo'), 'prompt_sentiment2'] = 'negative'
prompt.loc[prompt['prompt_sentiment'].str.contains('negativa'), 'prompt_sentiment2'] = 'negative'
prompt.loc[prompt['prompt_sentiment'].str.contains('neutro'), 'prompt_sentiment2'] = 'neutral'
prompt.loc[prompt['prompt_sentiment'].str.contains('neutra'), 'prompt_sentiment2'] = 'neutral'


In [110]:
prompt.head()

,chat_id,message,user,created_at,message_trans,prompt_sentiment,prompt_sentiment2
0,14VVFZN7R80UNPVA8Q04,Empresa Preciso cadastrsr outra empresan __aud...,Seoneco 🐸,2023-11-27T13:20:31.515216-03:00,company i need to register another company __a...,a análise do sentimento desse texto é neutra. ...,neutral
1,1DP3MDANB0OGQKASW1OM,O cadastro já deu certo agora só falta pedir a...,Welder Vendedor,2024-01-02T12:35:00.727364-03:00,"the registration has already been successful, ...",o sentimento do texto é negativo. o cliente pa...,negative
2,1OHYT505S3V28ELHD28U,Mais eu sou uma criança Como faço Eu sou uma c...,Tulio undefined,2023-11-30T11:02:34.54443-03:00,"desculpe, mas o texto fornecido é muito confus...",a análise do sentimento desse texto é negativa...,negative
3,25472dae-41be-4db5-b465-92b3ee0f285d,"Meu salário é 2000,00 e vou tirar 10 dias de f...",Thaissa Delgado,2023-12-04T19:42:38.730139-03:00,my salary is 2000.00 and i'm going to take 10 ...,o sentimento deste texto parece ser negativo. ...,negative
4,2S8DYFBPKE8253H7EFPH,O pagamento em 12x para o cliente tem juros? E...,Cidadedossonhosmoveis undefined,2023-12-07T14:08:58.011084-03:00,does the 12x payment for the customer have int...,"analisando o texto, o sentimento geral parece ...",positive


In [111]:
#conte quantos sentment foram positivos, negativos e neutros

prompt['prompt_sentiment2'].value_counts()

prompt_sentiment2
neutral     23
negative     5
positive     2
Name: count, dtype: int64

In [125]:
#mostre os negativos

prompt[df['prompt_sentiment2'] == 'negative']

,chat_id,message,user,created_at,message_trans,prompt_sentiment,prompt_sentiment2
1,1DP3MDANB0OGQKASW1OM,O cadastro já deu certo agora só falta pedir a...,Welder Vendedor,2024-01-02T12:35:00.727364-03:00,"the registration has already been successful, ...",o sentimento do texto é negativo. o cliente pa...,negative
2,1OHYT505S3V28ELHD28U,Mais eu sou uma criança Como faço Eu sou uma c...,Tulio undefined,2023-11-30T11:02:34.54443-03:00,"desculpe, mas o texto fornecido é muito confus...",a análise do sentimento desse texto é negativa...,negative
3,25472dae-41be-4db5-b465-92b3ee0f285d,"Meu salário é 2000,00 e vou tirar 10 dias de f...",Thaissa Delgado,2023-12-04T19:42:38.730139-03:00,my salary is 2000.00 and i'm going to take 10 ...,o sentimento deste texto parece ser negativo. ...,negative
5,3HX5TD1K48QBH4IDESXZ,"pode deixar, sua ajuda muito é de pouca avalia...",Fernando Bronté,2023-12-13T11:02:26.117422-03:00,"you can leave it, your help is of little value...",o sentimento expresso neste texto é negativo. ...,negative
12,CTCXA7V50J8NTO6LDQ8W,oi entendi e como funciona? qual a cor da cami...,Bernard W. Prado,2023-12-01T09:16:01.349879-03:00,"hi, i understood and how does it work? what is...",a análise do sentimento desse texto é negativa...,negative


In [130]:
#puxe de um texto inteiro da coluna prompt_sentment

prompt['prompt_sentiment'][2]

'a análise do sentimento desse texto é negativa. o cliente demonstra frustração e insatisfação com o atendimento, como evidenciado por palavras e frases como "vai caga" e "quantos você pra tapalha". além disso'

COMPARANDO

In [114]:
# Mesclando df_original com df_vaders usando chat_id como chave
comparativo = pd.merge(df[['chat_id', 'message', 'message_trans']], 
                       vaders[['chat_id', 'compound', 'vaders_sentiment']], 
                       on='chat_id', 
                       how='inner')

In [115]:
# Mesclando o resultado anterior com df_roberta
comparativo = pd.merge(comparativo, 
                       roberta[['chat_id', 'roberta_neg', 'roberta_neu', 'roberta_pos', 'sentiment_roberta']], 
                       on='chat_id', 
                       how='inner')

In [117]:
# Finalmente, mesclando com df_prompt
comparativo = pd.merge(comparativo, 
                       prompt[['chat_id', 'prompt_sentiment', 'prompt_sentiment2']], 
                       on='chat_id', 
                       how='inner')

In [118]:
# Criando uma nova DataFrame para as contagens
contagens = pd.DataFrame({
    'vader': comparativo['vaders_sentiment'].value_counts(),
    'roberta': comparativo['sentiment_roberta'].value_counts(),
    'prompt': comparativo['prompt_sentiment2'].value_counts()
})

# Reordenando as linhas para garantir a consistência entre as colunas
contagens = contagens.reindex(['positive', 'neutral', 'negative'])

In [119]:
contagens.head()

,vader,roberta,prompt
positive,24,3,2
neutral,4,23,23
negative,2,4,5


In [120]:
#mostre os negativos que saíram iguais nos 3 modelos

comparativo[(comparativo['vaders_sentiment'] == 'negative') & (comparativo['sentiment_roberta'] == 'negative') & (comparativo['prompt_sentiment2'] == 'negative')]

,chat_id,message,message_trans,compound,vaders_sentiment,roberta_neg,roberta_neu,roberta_pos,sentiment_roberta,prompt_sentiment,prompt_sentiment2
5,3HX5TD1K48QBH4IDESXZ,"pode deixar, sua ajuda muito é de pouca avalia...","you can leave it, your help is of little value...",-0.4304,negative,0.869413,0.116679,0.013909,negative,o sentimento expresso neste texto é negativo. ...,negative


In [121]:
#mostre os positivos que saíram iguais nos 3 modelos

comparativo[(comparativo['vaders_sentiment'] == 'positive') & (comparativo['sentiment_roberta'] == 'positive') & (comparativo['prompt_sentiment2'] == 'positive')]

,chat_id,message,message_trans,compound,vaders_sentiment,roberta_neg,roberta_neu,roberta_pos,sentiment_roberta,prompt_sentiment,prompt_sentiment2
4,2S8DYFBPKE8253H7EFPH,O pagamento em 12x para o cliente tem juros? E...,does the 12x payment for the customer have int...,0.8738,positive,0.007801,0.143044,0.849155,positive,"analisando o texto, o sentimento geral parece ...",positive


In [122]:
#mostre o texto completo da coluna "message" do chat_id 2S8DYFBPKE8253H7EFPH

comparativo[comparativo['chat_id'] == '2S8DYFBPKE8253H7EFPH']['message'].values[0]

'O pagamento em 12x para o cliente tem juros? E para cair na conta pra mim? a taça de antecipação para mim é quanto? obrigada Gostei,vou fazer Se eu quiser antecipar como funciona? E pagamento no cartão? A entendi O dinheiro pago em Pix por exemplo,fica na conta para eu usar da maneira que eu quiser ou só para comprar em fornecedor? Será que vc pode me explicar como funciona a Blu? Oi bruna'

In [123]:

#transforme o arquivo interactions_table_nov_jan.csv em um df6

file_path = 'interactions_table_nov_jan.csv'
df6 = pd.read_csv(file_path)